In [1]:
import os, sys, shutil
from xml.dom.minidom import Document

from dotenv import load_dotenv

load_dotenv("../../_apikeys.env")
api_key = os.getenv("DoogieOpenaiKey")
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
#
# Step 1-1: get test documents
#
import urllib.request

urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf", filename="06_07_test.pdf")

In [2]:
#
# Step 1-2: load and split documents
# 17~20 sec
#
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader( "06_07_test.pdf" )
pages = loader.load_and_split()  # about 17s
#pages = loader.load()  # about 16s, include empty page


In [ ]:
print( 'type loader,pages:', type(loader), type(pages) )
print( 'type pages[0]s:', type(pages[0]), type(pages[0].page_content))
print( 'size/len loader,pages:', sys.getsizeof(loader), len(pages) )
print( 'size/len pages[0]s:', sys.getsizeof(pages[0]), len(pages[0].page_content) )
print( 'pages Max:', max(len(apage.page_content) for apage in pages ) )
print( 'pages Min:', min(len(apage.page_content) for apage in pages ) )
print( 'pages Sum:', sum(len(apage.page_content) for apage in pages ) )
print( '#of chunks:', len(pages) )
print( 'Chunk Avg:', sum(len(apage.page_content) for apage in pages ) / len(pages) )

In [ ]:
#print( pages[0] )
#print( pages[0].page_content)
#print( "example:", pages[6].page_content[:50] )
#print( pages[442].page_content )

In [50]:
# Step 2: case1:
# RecursiveCharacterTextSpilter를 사용함
#
from langchain.text_splitter import RecursiveCharacterTextSplitter

textSplitter = RecursiveCharacterTextSplitter(
    chunk_size= 1000, # 1000
    chunk_overlap=0,
    separators=["\n\n", "\n", " ", ""]
)

splitedDocs = textSplitter.split_documents( pages )

In [ ]:
print( 'type split,splitDocs:', type(textSplitter), type(splitedDocs) )
print( 'type splitedDocs[0]s:', type(splitedDocs[0]), type(splitedDocs[0].page_content) )
print( 'size/len split,splitDocs:', sys.getsizeof(textSplitter), len(splitedDocs) )
print( 'size/len splitedDocs[0]s:', sys.getsizeof(splitedDocs[0]), len(splitedDocs[0].page_content) )
print( 'splitedDocs Max:', max(len(apage.page_content) for apage in splitedDocs ) )
print( 'splitedDocs Min:', min(len(apage.page_content) for apage in splitedDocs ) )
print( 'splitedDocs Sum:', sum(len(apage.page_content) for apage in splitedDocs ) )
print( '#of chunks:', len(splitedDocs) )
print( 'Chunk Avg:', sum(len(apage.page_content) for apage in splitedDocs ) / len(splitedDocs) )

In [ ]:
#
# DONT USE THIS CODE !!! use Step3-2 Code !!!
# Step3-1: Loading splitedDocs to chroma
# Error case: "Requested 367501 tokens, max 300000 tokens per request"
#
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

myEmbeddings = OpenAIEmbeddings()
print("current OpenAIEmbeddings().model=", myEmbeddings.model)
myEmbeddings = OpenAIEmbeddings(model="text-embedding-3-large")
print("current OpenAIEmbeddings().model=", myEmbeddings.model)

fdb = FAISS.from_documents(
    splitedDocs,
    myEmbeddings
)
print("vdb적재문서수=", fdb.index.ntotal)


In [51]:
#
# Step3-2: using package 'langchain_openai', not 'OpenAIEmbeddings'
#          from langchain.embeddings import OpenAIEmbeddings
#          'langchain.embeddings' deprecated, use 'langchain_openai'
#
#from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain.embeddings import OpenAIEmbeddings
#import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

myEmbeddings = OpenAIEmbeddings()
print("current OpenAIEmbeddings().model=", myEmbeddings.model)
myEmbeddings = OpenAIEmbeddings(model="text-embedding-3-large")
print("current OpenAIEmbeddings().model=", myEmbeddings.model)

faiss_index_path = "../localdb/my_faiss_db_11"
#myCollectionName = "my_collection"

if os.path.exists(faiss_index_path):
    print(f"기존 FAISS DB 삭제: {faiss_index_path}")
    shutil.rmtree(faiss_index_path)

#
# if u first create and add with splitedDocs to myEmbeddings, use Step3-2
# but if u want to use already Chroma data, use Step3-3
#


current OpenAIEmbeddings().model= text-embedding-ada-002
current OpenAIEmbeddings().model= text-embedding-3-large
기존 FAISS DB 삭제: ../localdb/my_faiss_db_11


In [52]:
#
# Step3-3: Open or ClearInit FAISS
#
from langchain.schema import Document
fdb = None

def create_open_faiss(persist_dir, embedding_func):
    global fdb

    print(f"[create_open_faiss] FAISS DB ({persist_dir})...")
    if os.path.exists(persist_dir):
        print(f"FAISS DB already exists, loading...: {persist_dir}")
        fdb = FAISS.load_local(
            folder_path = persist_dir,
            embeddings = embedding_func,
            allow_dangerous_deserialization=True,
        )
        print(f"FAISS DB loaded: {fdb.index.ntotal} documents...")
        return_val = fdb.index.ntotal
    else:
        print(f"FAISS DB does not exist, creating...: {persist_dir}")
        if len(splitedDocs) == 0:
            print("추가할 문서가 없습니다.")
            exit()

        # 첫 문서 1개로 초기화
        #fdb = FAISS.from_documents([splitedDocs[0]], myEmbeddings)
        #print("FAISS DB 초기화 완료 (with 첫 1개 문서)")

        # Dummy 문서 1개로 초기화
        dummy_doc = Document(page_content="dummy initialization",
                             metadata={"source": "init"})
        fdb = FAISS.from_documents([dummy_doc], embedding_func)
        dummy_id = list(fdb.docstore._dict.keys())[0]
        fdb.delete([dummy_id])
        print(f"빈 FAISS DB 생성 완료 {dummy_id}")
        return_val = 0

    print(f"[create_open_faiss] FAISS DB lendth: ({fdb.index.ntotal}) items...")
    fdb.save_local(faiss_index_path)
    return return_val

def clear_faiss(persist_dir):
    global fdb

    if fdb is not None and fdb.index.ntotal > 0:
        all_ids = list(fdb.docstore._dict.keys())
        if all_ids:
            fdb.delete(all_ids)
            print(f"[clear FAISS] 모든 문서 삭제 완료: {len(all_ids)}개")
            print(f"[clear FAISS] 삭제 후 크기: {fdb.index.ntotal}")
    else:
        print("[clear FAISS] 삭제할 문서가 없습니다.")


def batch_add_docs(persist_dir, documents, batch_size=100):
    global fdb

    splitedDocsLen = len(documents)
    print(">> ready to insert splitedDocs, len = ", splitedDocsLen)

    if splitedDocsLen == 0:
        print("splitedDocs(documents)가 비어있습니다.")
        return 0

    if fdb is None:
        print("FAISS DB가 NONE입니다...")
        return 0

    batchCount = splitedDocsLen // batch_size
    if splitedDocsLen % batch_size > 0:
        batchCount += 1
    print(">> batchsize is ", batch_size, "and batchCount:", batchCount )

    for i in range(0, splitedDocsLen, batch_size):
        #print(">>", i, "th batch, ", batch, "..", batch + batch_size - 1, end="")
        print(f">>add: { min(i + batch_size, splitedDocsLen)}/{splitedDocsLen}", end="")
        batchDocs = documents[i:i + batch_size]
        fdb.add_documents(batchDocs)
        print(" ++ added batchDocs len:", len(batchDocs))

    fdb.save_local(persist_dir)
    print(">> FAISS 인덱스 저장 완료, 위치: ", persist_dir)
    print(">> fdb size:", len(fdb.index_to_docstore_id) )
    print(">> fdb size:", fdb.index.ntotal )
    return


print(f"[START] FAISS DB ({faiss_index_path}) add documents...:")
count = create_open_faiss(faiss_index_path, myEmbeddings)
print(f"create_open_faiss() return_val={count}")
print(f"docstore_id={len(fdb.index_to_docstore_id)}, ntotal={fdb.index.ntotal}" )

if count == 0:
    batch_add_docs(faiss_index_path, splitedDocs, 100)

#
# optional: if u want to only clear exist FAISS
#
#clear_faiss(faiss_index_path)


[START] FAISS DB (../localdb/my_faiss_db_11) add documents...:
[create_open_faiss] FAISS DB (../localdb/my_faiss_db_11)...
FAISS DB does not exist, creating...: ../localdb/my_faiss_db_11
빈 FAISS DB 생성 완료 772d0e9d-4ce1-4aa0-9eda-6021189dc250
[create_open_faiss] FAISS DB lendth: (0) items...
create_open_faiss() return_val=0
docstore_id=0, ntotal=0
>> ready to insert splitedDocs, len =  496
>> batchsize is  100 and batchCount: 5
>>add: 100/496 ++ added batchDocs len: 100
>>add: 200/496 ++ added batchDocs len: 100
>>add: 300/496 ++ added batchDocs len: 100
>>add: 400/496 ++ added batchDocs len: 100
>>add: 496/496 ++ added batchDocs len: 96
>> FAISS 인덱스 저장 완료, 위치:  ../localdb/my_faiss_db_11
>> fdb size: 496
>> fdb size: 496


In [53]:
#
# Step3-4: open FAISS and use
#
faissdb = FAISS.load_local(
    folder_path = faiss_index_path,
    embeddings = myEmbeddings,
    allow_dangerous_deserialization=True,
)

print(faissdb.index.ntotal)
print(faissdb.index.d)
print(faissdb.index.is_trained)


496
3072
True


In [44]:
#
# Step4-1:
#
question = '북한의 교육과정'
resultDocs = fdb.similarity_search(question)
print('유사문서수:', len(resultDocs))
print( type(resultDocs), type(resultDocs[0]), type(resultDocs[0].page_content) )

유사문서수: 4
<class 'list'> <class 'langchain_core.documents.base.Document'> <class 'str'>


In [45]:
for resultDoc in resultDocs:
    print('---' * 10)
    print(resultDoc.metadata.get("page_label"), resultDoc.page_content[:100] )


------------------------------
351 4. 교육권
349
IV. 경제적·사회적·문화적 권리 I. 발간개요V. 취약계층VI. 특별사안 II. 요약III. 시민적·정치적 권리
및 종교 집단 사이에 이해를 증진시킬 수 있도
------------------------------
352 2023 북한인권보고서
350
소학교 때는 김일성, 김정일, 김정숙의 어린 시절을 배우고, 초급중
학교에서는 혁명활동을 배우는데, 김정숙과 김정은의 내용을 학기
마다 번갈아 가며 
------------------------------
42 2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에
------------------------------
339 4. 교육권
337
IV. 경제적·사회적·문화적 권리 I. 발간개요V. 취약계층VI. 특별사안 II. 요약III. 시민적·정치적 권리
예산으로 보장하며,338 교과서나 교육비품의 


In [48]:
#
# 4-2: other handle of FAISS DB
#
fdb2 = FAISS.load_local(
    folder_path = faiss_index_path,
    embeddings = myEmbeddings,
    allow_dangerous_deserialization=True,
)
print('문서의 수:', fdb2.index.ntotal)
print( type(fdb), type(fdb2) )

question = '북한의 교육과정'
resultDocs = fdb2.similarity_search(question)
print('유사문서수:', len(resultDocs))
print( type(resultDocs), type(resultDocs[0]), type(resultDocs[0].page_content) )

for resultDoc in resultDocs:
    print('---' * 10)
    print(resultDoc.metadata.get("page_label"), resultDoc.page_content[:100] )


문서의 수: 445
<class 'langchain_community.vectorstores.faiss.FAISS'> <class 'langchain_community.vectorstores.faiss.FAISS'>
유사문서수: 4
<class 'list'> <class 'langchain_core.documents.base.Document'> <class 'str'>
------------------------------
351 4. 교육권
349
IV. 경제적·사회적·문화적 권리 I. 발간개요V. 취약계층VI. 특별사안 II. 요약III. 시민적·정치적 권리
및 종교 집단 사이에 이해를 증진시킬 수 있도
------------------------------
352 2023 북한인권보고서
350
소학교 때는 김일성, 김정일, 김정숙의 어린 시절을 배우고, 초급중
학교에서는 혁명활동을 배우는데, 김정숙과 김정은의 내용을 학기
마다 번갈아 가며 
------------------------------
42 2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에
------------------------------
339 4. 교육권
337
IV. 경제적·사회적·문화적 권리 I. 발간개요V. 취약계층VI. 특별사안 II. 요약III. 시민적·정치적 권리
예산으로 보장하며,338 교과서나 교육비품의 


In [54]:
#
# 4-3
# find top 3, print similarity score
# List[Document] -> docs[0].metadata, docs[0].page_content
# List[Tuple[Document, float]] -> docs[0][0].metadata, docs[0][1]
question = '북한의 교육과정'
resultDocs = fdb2.similarity_search_with_relevance_scores(question, k=3)
print('유사문서수:', len(resultDocs))
print( type(resultDocs), type(resultDocs[0]) )
print( type(resultDocs[0][0]), type(resultDocs[0][0].page_content) )
print( type(resultDocs[0][1]), resultDocs[0][1] )

for resultDoc in resultDocs:
    print('---' * 10)
    print(resultDoc[1], resultDoc[0].page_content[:100])

for resultDoc, score in resultDocs:
    print('---' * 10)
    print(score, len(resultDoc.page_content) )

유사문서수: 3
<class 'list'> <class 'tuple'>
<class 'langchain_core.documents.base.Document'> <class 'str'>
<class 'numpy.float32'> 0.36758494
------------------------------
0.36758494 4. 교육권
349
IV. 경제적·사회적·문화적 권리 I. 발간개요V. 취약계층VI. 특별사안 II. 요약III. 시민적·정치적 권리
및 종교 집단 사이에 이해를 증진시킬 수 있도
------------------------------
0.36240548 2023 북한인권보고서
350
소학교 때는 김일성, 김정일, 김정숙의 어린 시절을 배우고, 초급중
학교에서는 혁명활동을 배우는데, 김정숙과 김정은의 내용을 학기
마다 번갈아 가며 
------------------------------
0.36217612 2023 북한인권보고서
40
명목의 교육비용이 전가되고 있는 것으로 나타났다. 교과서는 ‘교과
서 요금’이라는 명목으로 일정 금액을 내야하는 경우가 많으며, 교
과서가 모든 학생에
------------------------------
0.36758494 1157
------------------------------
0.36240548 867
------------------------------
0.36217612 866
